In [496]:
from IPython.display import IFrame
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches
import warnings
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import zipfile

from IPython.display import IFrame
from shapely.geometry import Point, LineString, shape, Polygon


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Testing Out GTFS Data Visualization using NYC Ferry GTFS
## NYC Ferry GTFS Contains:
- agency.txt
- routes.txt
- shapes.txt
- stops.txt
- trips.txt

With the above files from any GTFS data, I should be able to produce a quick transit map of any city which produces GTFS data.
Reding in all data here:

In [497]:
city_name = 'alexandria'
z = zipfile.ZipFile(f"{city_name}_gtfs.zip")
z.extractall(f"{city_name}_gtfs")

In [498]:
#Read in all necessary data from GTFS folder
agency = pd.read_csv(f"{city_name}_gtfs/agency.txt")
routes = pd.read_csv(f"{city_name}_gtfs/routes.txt")
shapes = pd.read_csv(f"{city_name}_gtfs/shapes.txt")
stops = pd.read_csv(f"{city_name}_gtfs/stops.txt")
trips = pd.read_csv(f"{city_name}_gtfs/trips.txt")

In [499]:
# Taking agency name to save html file at the end.
html_name = agency['agency_name'][0]

In [500]:
html_name

'DASH'




## Mapping the Stops
GTFS Data contains coordinates for stops along all the lines.

In [501]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1181 entries, 0 to 1180
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trip_id                1181 non-null   int64  
 1   route_id               1181 non-null   object 
 2   service_id             1181 non-null   int64  
 3   trip_headsign          1181 non-null   object 
 4   trip_short_name        0 non-null      float64
 5   direction_id           1181 non-null   int64  
 6   block_id               1181 non-null   int64  
 7   shape_id               1181 non-null   object 
 8   wheelchair_accessible  1181 non-null   int64  
 9   bikes_allowed          1181 non-null   int64  
 10  block_name             0 non-null      float64
dtypes: float64(2), int64(6), object(3)
memory usage: 101.6+ KB


In [502]:
#read in stops as gdf
stops_gdf = gpd.GeoDataFrame(
    stops,
    crs=4326,
    geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat))




## Mapping the Routes

In order to map out the routes in a line, first I will need to convert points from shape.txt, merge routes.txt for route name and other info.

In [503]:
# Create GeoDataFrame from Shapes.txt
shapes_gdf = gpd.GeoDataFrame(
    shapes,
    crs=4326,
    geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat)
)

In [504]:
# Create Linestring out of shape_id and shape_pt_sequence
shapes_gdf = shapes_gdf.groupby(['shape_id'])['geometry'].apply(lambda x: LineString(x.tolist()))
shapes_gdf = gpd.GeoDataFrame(shapes_gdf, geometry='geometry')

In [505]:
#keeping trips info only for each trips to take out trip_id
trips_drop = trips.drop_duplicates(subset=['shape_id'])

In [506]:
#merging trip_id on to shapes in order to merge into route.txt
shape_merge = shapes_gdf.merge(trips_drop,left_on='shape_id', right_on='shape_id', how='outer')

In [507]:
#dropping duplicate shapes to simplify route for this test
shape_merge = shape_merge.drop_duplicates(subset=['route_id'])

In [508]:
#Route lines merged with name, routes.txt
shape_routes = shape_merge.merge(routes, left_on='route_id', right_on='route_id', how='inner')

In [509]:
#shape_routes.crs = {'init' :'epsg:4326'}

## Preparing datasets to publish on a map

### Converting files into GeoJson

In [510]:
#stops, convert to 4326 in case it was in some other format
sjson = stops_gdf.to_json()

In [511]:
#routes converted to GeoJson
rjson = shape_routes.to_json()

In [512]:
shape_routes.head()

,shape_id,geometry,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,block_name,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,shp-1-02,"LINESTRING (-77.12875 38.79996, -77.12883 38.7...",470070,AT1+,2,Seminary Plaza,NaN,1,195402707,0,0,NaN,1,AT1+,West End-Landmark Plaza-Beauregard St,NaN,3,NaN,00b0ee,000000
1,shp-10-01,"LINESTRING (-77.05913 38.80605, -77.05929 38.8...",548070,AT10,2,Potomac Yard Center,NaN,1,195508207,0,0,NaN,1,AT10,Del Ray-Potomac Yard,NaN,3,NaN,6508e3,ffffff
2,shp-2-01,"LINESTRING (-77.13799 38.81834, -77.13793 38.8...",632020,AT2,1,Braddock Metro,NaN,1,195634202,0,0,NaN,1,AT2,Old Town-Seminary Rd-Beauregard St,NaN,3,NaN,e46c0a,ffffff
3,shp-3-09,"LINESTRING (-77.05351 38.81412, -77.05357 38.8...",74020,AT3,1,Pentagon Metro via Parkfairfax,NaN,1,195739502,0,0,NaN,1,AT3,South Old Town-Russell Rd-W Glebe Rd,NaN,3,NaN,ff5c90,000000
4,shp-34-04,"LINESTRING (-77.04459 38.79975, -77.04467 38.7...",537020,AT3-4,1,ParkFairfax - Old Town/Royal&Wilkes,NaN,1,195739402,0,0,NaN,1,AT3-4,Old Town-Parkfairfax Loop,NaN,3,NaN,ff0014,ffffff


## Mapping Options

In [513]:
# Function identifying centroid of routes to display original location of map
def get_feature_centroid(gdf):
    gdf['dissolve'] = 1
    center = gdf.dissolve(by='dissolve').centroid
    return center.x, center.y

In [516]:
# Getting Sentriod to display original coordinates for map.
x, y = get_feature_centroid(shape_routes)

# Setting up Basemap
m = folium.Map(
    location=[y, x],
    zoom_start=10,
    tiles='cartodbpositron') 

# Function to return route color with # in front to display properly for leaflet style_function
def style_geojson(features):
    #if features['properties']['route_color'] == True:
        return {'color': f"#{features['properties']['route_color']}"} 
    #else:
    #    return {'color': 'blue'}
    
# Display GeoJson of Routes
folium.GeoJson(
    rjson,
    name= 'Routes',
    style_function=style_geojson,
).add_to(m)

# Creating and displaying folium.Marker for every stop
for index,row in stops_gdf.iterrows():
    x, y = row['geometry'].centroid.x, row['geometry'].centroid.y
    folium.Circle(
        location = [y, x],
        radius=20,
        popup=" Station Name: " + row['stop_name'],
        color='black',
        fill=True,
        fill_opacity=500,
    ).add_to(m)
m

## Creating HTML File for Map

In [382]:
m.save(f"{html_name}.html")